# Colab Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"""
Change directory to where this file is located
"""
%cd '/content/drive/MyDrive/MLDL1/HW3'


/content/drive/MyDrive/MLDL1/HW3


# Import Modules

In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt
from mnist.data_utils import load_data

#Utils

In [ ]:
def leaky_relu(z):
    """
    Implement the leaky ReLU activation function.
    The method takes the input z and returns the output of the function.
    
    Set the value of alpha for the leaky ReLU funtion to 0.01.
    Question (a)

    """
    #### YOUR CODE #####
    return np.maximum(0.01 * z, z)
    #####################

def softmax(X):
    """
    Implement the softmax function.
    The method takes the input X and returns the output of the function.

    Question (a)

    """
    ##### YOUR CODE #####
    X_shifted = X - np.max(X, axis=1, keepdims=True)
    exp_X = np.exp(X_shifted)
    return exp_X / np.sum(exp_X, axis=1, keepdims=True) # np.exp(x - np.max(x))
    #####################

def load_batch(X, Y, batch_size, shuffle=True):
    """
    Generates batches with the remainder dropped.

    Do NOT modify this function
    """
    if shuffle:
        permutation = np.random.permutation(X.shape[0])
        X = X[permutation, :]
        Y = Y[permutation, :]
    num_steps = int(X.shape[0])//batch_size
    step = 0
    while step<num_steps:
        X_batch = X[batch_size*step:batch_size*(step+1)]
        Y_batch = Y[batch_size*step:batch_size*(step+1)]
        step+=1
        yield X_batch, Y_batch

#2-Layer Neural Network

In [ ]:
class TwoLayerNN:
    """ a neural network with 2 layers """

    def __init__(self, input_dim, num_hiddens, num_classes):
        """
        Do NOT modify this function.
        """
        self.input_dim = input_dim
        self.num_hiddens = num_hiddens
        self.num_classes = num_classes
        self.params = self.initialize_parameters(input_dim, num_hiddens, num_classes)

    def initialize_parameters(self, input_dim, num_hiddens, num_classes):
        """
        initializes parameters with He Initialization.

        Question (b)
        - refer to https://paperswithcode.com/method/he-initialization for He initialization 
        
        Inputs
        - input_dim
        - num_hiddens
        - num_classes
        Returns
        - params: a dictionary with the initialized parameters.
        """
        params = {}
        ##### YOUR CODE #####
        params["W1"] = np.random.randn(num_hiddens,input_dim)*np.sqrt(2/input_dim)
        params["b1"] = np.zeros((num_hiddens,1))
        params["W2"] = np.random.randn(num_classes,num_hiddens) * np.sqrt(2/num_hiddens)  
        params["b2"] = np.zeros((num_classes,1))
        #####################
        
        return params

    def forward(self, X):
        """
        Define and perform the feed forward step of a two-layer neural network.
        Specifically, the network structue is given by

          y = softmax(leaky_relu(X W1 + b1) W2 + b2)

        where X is the input matrix of shape (N, D), y is the class distribution matrix
        of shape (N, C), N is the number of examples (either the entire dataset or
        a mini-batch), D is the feature dimensionality, and C is the number of classes.

        Question (c)
        - ff_dict will be used to run backpropagation in backward method.

        Inputs
        - X: the input matrix of shape (N, D)

        Returns
        - y: the output of the model
        - ff_dict: a dictionary with all the fully connected units and activations.
        """
        ff_dict = {}
        ##### YOUR CODE #####
        ff_dict["z1"] = np.dot(X, self.params["W1"].T) + self.params["b1"].T 
        ff_dict["a1"] = leaky_relu(ff_dict["z1"]) 
        ff_dict["z2"] = np.dot(ff_dict["a1"], self.params["W2"].T) + self.params["b2"].T 
        y = softmax(ff_dict["z2"]) 
        ff_dict["y"] = y
        #####################
        return y, ff_dict

    def backward(self, X, Y, ff_dict):
        """
        Performs backpropagation over the two-layer neural network, and returns
        a dictionary of gradients of all model parameters.

        Question (d)

        Inputs:
         - X: the input matrix of shape (B, D), where B is the number of examples
              in a mini-batch, D is the feature dimensionality.
         - Y: the matrix of one-hot encoded ground truth classes of shape (B, C),
              where B is the number of examples in a mini-batch, C is the number
              of classes.
         - ff_dict: the dictionary containing all the fully connected units and
              activations.

        Returns:
         - grads: a dictionary containing the gradients of corresponding weights and biases.
        """
        grads = {}
        ##### YOUR CODE #####
        '''
        grads["dW1"] = None
        grads["db1"] = None
        grads["dW2"] = None
        grads["db2"] = None
        '''

        # Compute gradients of the last layer
        dZ2 = (ff_dict["y"] - Y) 
        grads["dW2"] = np.dot(ff_dict["a1"].T, dZ2).T 
        grads["db2"] = np.sum(dZ2, axis=0).reshape(-1,1)

        # Compute gradients of the hidden layer
        dZ1 = np.dot(dZ2, self.params["W2"])*np.where(ff_dict['z1'] >=0, 1, 0.01)
        grads["dW1"] = np.dot(dZ1.T, X)
        grads["db1"] = np.sum(dZ1, axis=0).reshape(-1,1)

        ##############
        return grads

    def compute_loss(self, Y, Y_hat):
        """
        Computes cross entropy loss.

        Do NOT modify this function.

        Inputs
            Y:
            Y_hat:
        Returns
            loss:
        """
        epsilon = 1e-10
        Y_hat = np.clip(Y_hat, epsilon, 1 - epsilon)
        loss = -(1/Y.shape[0]) * np.sum(np.multiply(Y, np.log(Y_hat)))
        return loss

    def train(self, X, Y, X_val, Y_val, lr, n_epochs, batch_size, log_interval=1):
        """
        Runs mini-batch gradient descent.

        Do NOT Modify this method.

        Inputs
        - X
        - Y
        - X_val
        - Y_Val
        - lr
        - n_epochs
        - batch_size
        - log_interval
        """
        for epoch in range(n_epochs):
            for X_batch, Y_batch in load_batch(X, Y, batch_size):
                self.train_step(X_batch, Y_batch, batch_size, lr)
            if epoch % log_interval==0:
                Y_hat, ff_dict = self.forward(X)
                train_loss = self.compute_loss(Y, Y_hat)
                train_acc = self.evaluate(Y, Y_hat)
                Y_hat, ff_dict = self.forward(X_val)
                valid_loss = self.compute_loss(Y_val, Y_hat)
                valid_acc = self.evaluate(Y_val, Y_hat)
                print('epoch {:02} - train loss/acc: {:.3f} {:.3f}, valid loss/acc: {:.3f} {:.3f}'.\
                      format(epoch, train_loss, train_acc, valid_loss, valid_acc))

    def train_step(self, X_batch, Y_batch, batch_size, lr):
        """
        Updates the parameters using gradient descent.

        Do NOT Modify this method.

        Inputs
        - X_batch
        - Y_batch
        - batch_size
        - lr
        """
        _, ff_dict = self.forward(X_batch)
        grads = self.backward(X_batch, Y_batch, ff_dict)
        self.params["W1"] -= lr * grads["dW1"]/batch_size
        self.params["b1"] -= lr * grads["db1"]/batch_size
        self.params["W2"] -= lr * grads["dW2"]/batch_size
        self.params["b2"] -= lr * grads["db2"]/batch_size

    def evaluate(self, Y, Y_hat):
        """
        Computes classification accuracy.
        
        Do NOT modify this function

        Inputs
        - Y: A numpy array of shape (N, C) containing the softmax outputs,
             where C is the number of classes.
        - Y_hat: A numpy array of shape (N, C) containing the one-hot encoded labels,
             where C is the number of classes.

        Returns
            accuracy: the classification accuracy in float
        """        
        classes_pred = np.argmax(Y_hat, axis=1)
        classes_gt = np.argmax(Y, axis=1)
        accuracy = float(np.sum(classes_pred==classes_gt)) / Y.shape[0]
        return accuracy

#Load MNIST

In [ ]:
X_train, Y_train, X_test, Y_test = load_data()

idxs = np.arange(len(X_train))
np.random.shuffle(idxs)
split_idx = int(np.ceil(len(idxs)*0.8))
X_valid, Y_valid = X_train[idxs[split_idx:]], Y_train[idxs[split_idx:]]
X_train, Y_train = X_train[idxs[:split_idx]], Y_train[idxs[:split_idx]]
print()
print('Set validation data aside')
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', Y_train.shape)
print('Validation data shape: ', X_valid.shape)
print('Validation labels shape: ', Y_valid.shape)

MNIST data loaded:
Training data shape: (60000, 784)
Training labels shape: (60000, 10)
Test data shape: (10000, 784)
Test labels shape: (10000, 10)

Set validation data aside
Training data shape:  (48000, 784)
Training labels shape:  (48000, 10)
Validation data shape:  (12000, 784)
Validation labels shape:  (12000, 10)


#Training & Evaluation

In [ ]:
### 
# Question (e)
# Tune the hyperparameters with validation data, 
# and print the results by running the lines below.
###

In [ ]:
# model instantiation
model = TwoLayerNN(input_dim=784, num_hiddens=64, num_classes=10)

In [ ]:
# train the model
# I have tried multiple sets of combination, and concluded with the below hyperparameters.
lr, n_epochs, batch_size = 0.02, 20, 256 # changed the hyperparameters. (lr from 3-> 0.02)
model.train(X_train, Y_train, X_valid, Y_valid, lr, n_epochs, batch_size)

epoch 00 - train loss/acc: 0.788 0.828, valid loss/acc: 0.790 0.830
epoch 01 - train loss/acc: 0.532 0.870, valid loss/acc: 0.537 0.869
epoch 02 - train loss/acc: 0.446 0.883, valid loss/acc: 0.450 0.882
epoch 03 - train loss/acc: 0.401 0.893, valid loss/acc: 0.405 0.891
epoch 04 - train loss/acc: 0.373 0.899, valid loss/acc: 0.377 0.897
epoch 05 - train loss/acc: 0.354 0.902, valid loss/acc: 0.359 0.899
epoch 06 - train loss/acc: 0.338 0.906, valid loss/acc: 0.343 0.904
epoch 07 - train loss/acc: 0.326 0.908, valid loss/acc: 0.331 0.907
epoch 08 - train loss/acc: 0.315 0.911, valid loss/acc: 0.321 0.909
epoch 09 - train loss/acc: 0.306 0.914, valid loss/acc: 0.313 0.912
epoch 10 - train loss/acc: 0.299 0.916, valid loss/acc: 0.306 0.914
epoch 11 - train loss/acc: 0.292 0.917, valid loss/acc: 0.299 0.916
epoch 12 - train loss/acc: 0.285 0.920, valid loss/acc: 0.293 0.918
epoch 13 - train loss/acc: 0.279 0.922, valid loss/acc: 0.287 0.920
epoch 14 - train loss/acc: 0.274 0.923, valid lo

In [ ]:
# evalute the model on test data
Y_hat, _ = model.forward(X_test)
test_loss = model.compute_loss(Y_test, Y_hat)
test_acc = model.evaluate(Y_test, Y_hat)
print("Final test loss = {:.3f}, acc = {:.3f}".format(test_loss, test_acc))

Final test loss = 0.246, acc = 0.932
